i was surprised to be able to get 1.11 with linear regression, which is not bad, this also ensembled well with the boosted trees submissions, i got 1.00x doing this

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt

import gc
import pickle as pickle


from sklearn.model_selection import KFold
from itertools import product
import tensorflow as tf

In [7]:
gc.collect()

items           = pd.read_csv('items.csv',usecols=["item_id", "item_category_id"])
item_categories = pd.read_csv('item_categories.csv')
shops           = pd.read_csv('shops.csv')
sales_train     = pd.read_csv('sales_train.csv.gz')
test            = pd.read_csv('test.csv.gz')

In [8]:
sales_train[['day','month', 'year']] = sales_train['date'].str.split('.', expand=True).astype(int)
#various experiments, this worked better with very fresh data
sales_train = sales_train[sales_train['year'].isin([2013,2014]) == False]
sales_train = sales_train.set_index('item_id').join(items.set_index('item_id'))
sales_train.reset_index(inplace=True)

In [9]:
train_item_ids = sales_train['item_id'].unique()

train_shop_ids = sales_train['shop_id'].unique()
test_item_ids = test['item_id'].unique()
test_shop_ids = test['shop_id'].unique()
train_blocks = sales_train['date_block_num'].unique()

#same as the other notebooks
all_item_ids = test_item_ids

all_shop_ids = test_shop_ids



In [10]:

#same as the other notebooks

combinations = []

for dbn in range(np.min(train_blocks), np.max(train_blocks)+1):
    sales = sales_train[sales_train.date_block_num==dbn]
    #item_ids = np.intersect1d(sales.item_id.unique(), test_item_ids)
    item_ids = all_item_ids
    #dbn_combos = list(product(sales.shop_id.unique(), item_ids, [dbn]))
    dbn_combos = list(product(all_shop_ids, item_ids, [dbn]))
    for combo in dbn_combos:
        combinations.append(combo)
        
all_combos = pd.DataFrame(np.unique(np.vstack([combinations]), axis=0), columns=['shop_id','item_id','date_block_num'])

In [11]:
#same as the other notebooks


ys = sales_train.groupby(['shop_id', 'item_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"shop_item_cnt_block"})

training = all_combos.merge(ys, on=['shop_id', 'item_id', 'date_block_num'], how='left').fillna(0)


training['shop_item_cnt_block'] = training['shop_item_cnt_block'].clip(0,20).astype('int8')

training = training.set_index('item_id').join(items.set_index('item_id'))
training.reset_index(inplace=True)

for col in ['item_id', 'shop_id', 'item_category_id']:
    training[col] = pd.to_numeric(training[col], downcast='unsigned')

In [12]:
#same as the other notebooks


dates = sales_train[['date_block_num', 'month', 'year']].drop_duplicates(['date_block_num', 'month', 'year'])

dates_dict = {}

for index,row in dates.iterrows():
    dates_dict[row['date_block_num']] = {"month": row['month'], "year": row['year']}
    
training['month'] = pd.to_numeric(training['date_block_num'].apply(lambda block: dates_dict[block]['month']), downcast='unsigned')
training['year'] = pd.to_numeric(training['date_block_num'].apply(lambda block: dates_dict[block]['year']), downcast='unsigned')



In [13]:
#same as the other notebooks


ys = sales_train.groupby(['item_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"item_cnt_block"})

training = training.merge(ys, on=['item_id', 'date_block_num'], how='left').fillna(0)

ys = sales_train.groupby(['shop_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"shop_cnt_block"})

training = training.merge(ys, on=['shop_id', 'date_block_num'], how='left').fillna(0)


ys = sales_train.groupby(['item_category_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"category_cnt_block"})


training = training.merge(ys, on=['item_category_id', 'date_block_num'], how='left').fillna(0)


ys = sales_train.groupby(['shop_id', 'item_category_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"shop_category_cnt_block"})

training = training.merge(ys, on=['shop_id', 'item_category_id', 'date_block_num'], how='left').fillna(0)


In [ ]:
training['item_cnt_block_mean'] = training.groupby(['date_block_num'])['item_cnt_block'].transform(np.mean)
training['item_cnt_block_min'] = training.groupby(['date_block_num'])['item_cnt_block'].transform(np.min)
training['item_cnt_block_max'] = training.groupby(['date_block_num'])['item_cnt_block'].transform(np.max)
training['item_cnt_block_std'] = training.groupby(['date_block_num'])['item_cnt_block'].transform(np.std)
training['item_cnt_block_med'] = training.groupby(['date_block_num'])['item_cnt_block'].transform(np.median)

In [ ]:
training['shop_cnt_block_mean'] = training.groupby(['date_block_num'])['shop_cnt_block'].transform(np.mean)
training['shop_cnt_block_min'] = training.groupby(['date_block_num'])['shop_cnt_block'].transform(np.min)
training['shop_cnt_block_max'] = training.groupby(['date_block_num'])['shop_cnt_block'].transform(np.max)
training['shop_cnt_block_std'] = training.groupby(['date_block_num'])['shop_cnt_block'].transform(np.std)
training['shop_cnt_block_med'] = training.groupby(['date_block_num'])['shop_cnt_block'].transform(np.median)

In [ ]:
training['category_cnt_block_mean'] = training.groupby(['date_block_num'])['category_cnt_block'].transform(np.mean)
training['category_cnt_block_min'] = training.groupby(['date_block_num'])['category_cnt_block'].transform(np.min)
training['category_cnt_block_max'] = training.groupby(['date_block_num'])['category_cnt_block'].transform(np.max)
training['category_cnt_block_std'] = training.groupby(['date_block_num'])['category_cnt_block'].transform(np.std)
training['category_cnt_block_med'] = training.groupby(['date_block_num'])['category_cnt_block'].transform(np.median)

In [ ]:
training['shop_category_cnt_block_mean'] = training.groupby(['date_block_num'])['shop_category_cnt_block'].transform(np.mean)
training['shop_category_cnt_block_min'] = training.groupby(['date_block_num'])['shop_category_cnt_block'].transform(np.min)
training['shop_category_cnt_block_max'] = training.groupby(['date_block_num'])['shop_category_cnt_block'].transform(np.max)
training['shop_category_cnt_block_std'] = training.groupby(['date_block_num'])['shop_category_cnt_block'].transform(np.std)
training['shop_category_cnt_block_med'] = training.groupby(['date_block_num'])['shop_category_cnt_block'].transform(np.median)

In [ ]:
training['shop_item_cnt_block_mean'] = training.groupby(['date_block_num'])['shop_item_cnt_block'].transform(np.mean)
training['shop_item_cnt_block_min'] = training.groupby(['date_block_num'])['shop_item_cnt_block'].transform(np.min)
training['shop_item_cnt_block_max'] = training.groupby(['date_block_num'])['shop_item_cnt_block'].transform(np.max)
training['shop_item_cnt_block_std'] = training.groupby(['date_block_num'])['shop_item_cnt_block'].transform(np.std)
training['shop_item_cnt_block_med'] = training.groupby(['date_block_num'])['shop_item_cnt_block'].transform(np.median)

In [14]:
#https://maxhalford.github.io/blog/target-encoding-done-the-right-way/
#https://www.kaggle.com/vprokopev/mean-likelihood-encodings-a-comprehensive-study

#same as the other notebooks

from sklearn.model_selection import StratifiedKFold
columns = ["item_id", "shop_id", "item_category_id"]



y_train = training["shop_item_cnt_block"].values
folds = KFold(n_splits = 5, shuffle=True).split(training)

i=1
for in_fold_index, out_of_fold_index in folds:
    print("fold", i)
    #print(np.intersect1d(training.loc[in_fold_index]["shop_id"].unique(), training.loc[out_of_fold_index]["shop_id"].unique()))
    #print(len(in_fold_index))
    for column in columns:
        means = training.iloc[in_fold_index].groupby(column)['shop_item_cnt_block'].mean()
            #x_validation[column + "_mean_target"] = means\
        name = column + '_mean_encoding'
        training.loc[out_of_fold_index,name] = training.loc[out_of_fold_index][column].map(means)
    i+=1

fold 1
fold 2
fold 3
fold 4
fold 5


In [16]:
#i use a weighted average of the data, the fresher it is the heavier the weight so that the mean skews towards
#the most recent

weights=[1/55,2/55,3/55,4/55,5/55,6/55,7/55,8/55,9/55,10/55]
item_cnt_block_avg = training.groupby(['item_id', 'shop_id'])['item_cnt_block'].apply(np.average, weights=weights).reset_index().rename(columns={"item_cnt_block":"item_cnt_block_avg"})
print(1)
shop_cnt_block_avg = training.groupby(['item_id', 'shop_id'])['shop_cnt_block'].apply(np.average, weights=weights).reset_index().rename(columns={"shop_cnt_block":"shop_cnt_block_avg"})
print(2)
category_cnt_block_avg = training.groupby(['item_id', 'shop_id'])['category_cnt_block'].apply(np.average, weights=weights).reset_index().rename(columns={"category_cnt_block":"category_cnt_block_avg"})
print(3)
shop_cat_cnt_block_avg = training.groupby(['item_id', 'shop_id'])['shop_category_cnt_block'].apply(np.average, weights=weights)\
                    .reset_index().rename(columns={"shop_category_cnt_block":"shop_category_cnt_block_avg"})

1
2
3


In [17]:
#the average data is the building block of the train data

item_cnt_block_avg.head()
consolidated = item_cnt_block_avg
cols= ["item_id", "shop_id"]
consolidated = consolidated.merge(shop_cnt_block_avg,on=cols,how='left')
del item_cnt_block_avg
del shop_cnt_block_avg
consolidated = consolidated.merge(category_cnt_block_avg,on=cols,how='left')
del category_cnt_block_avg
consolidated = consolidated.merge(shop_cat_cnt_block_avg,on=cols,how='left')
del shop_cat_cnt_block_avg

In [24]:
len(consolidated)

214200

In [18]:
training.fillna(0,inplace=True)

In [23]:
consolidated = consolidated.merge(training[training['date_block_num']==33][cols+['date_block_num','shop_item_cnt_block']],on=cols,how='left')

In [26]:
consolidated.columns.values

array(['item_id', 'shop_id', 'item_cnt_block_avg', 'shop_cnt_block_avg',
       'category_cnt_block_avg', 'shop_category_cnt_block_avg',
       'date_block_num', 'shop_item_cnt_block'], dtype=object)

In [27]:
#also worked best with the basic feature set
features = [
    'item_cnt_block_avg', 'shop_cnt_block_avg',
       'category_cnt_block_avg', 'shop_category_cnt_block_avg'
]

In [28]:
#lr needs scaling

from sklearn.preprocessing import StandardScaler,MinMaxScaler 


consolidated[features] = MinMaxScaler().fit_transform(consolidated[features])

consolidated[features] = consolidated[features].apply(pd.to_numeric, downcast='float')

In [29]:
consolidated.sample(10)

,item_id,shop_id,item_cnt_block_avg,shop_cnt_block_avg,category_cnt_block_avg,shop_category_cnt_block_avg,date_block_num,shop_item_cnt_block
113665,12020,21,0.030665,0.278043,1.000000,0.153721,33,1
95787,10148,42,0.002291,0.639257,0.696913,0.213149,33,0
39240,4049,19,0.005858,0.219493,0.336978,0.034315,33,0
50451,5044,15,0.002567,0.200639,0.336978,0.042265,33,1
141005,14362,18,0.000311,0.192290,0.042667,0.015147,33,0
207641,21377,52,0.004235,0.141664,1.000000,0.079680,33,0
119489,12697,59,0.002090,0.146951,0.106027,0.008882,33,0
59706,6041,38,0.000301,0.204317,0.042667,0.000702,33,0
72411,7459,5,0.002909,0.161951,0.696913,0.078711,33,0
23795,2778,37,0.001372,0.147785,0.725807,0.098184,33,0


In [19]:

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
training.sample(10)

,item_id,shop_id,date_block_num,shop_item_cnt_block,item_category_id,month,year,item_cnt_block,shop_cnt_block,category_cnt_block,shop_category_cnt_block,item_id_mean_encoding,shop_id_mean_encoding,item_category_id_mean_encoding
296677,3245,24,31,0,24,8,2015,2.0,1186.0,1011.0,22.0,0.067901,0.205065,0.232087
1989398,20462,42,32,2,72,9,2015,7.0,4237.0,1258.0,103.0,0.173021,0.547448,0.143718
843651,8778,45,25,1,40,2,2015,17.0,796.0,12100.0,69.0,0.288288,0.118733,0.188009
1662543,16366,28,27,0,31,4,2015,0.0,3686.0,2506.0,0.0,0.021148,0.584290,0.044466
653644,6615,19,28,1,19,5,2015,19.0,1459.0,3392.0,78.0,0.290909,0.244068,0.534379
485852,4906,49,26,1,19,3,2015,40.0,679.0,3737.0,44.0,0.483965,0.129946,0.532307
984858,10347,55,32,0,40,9,2015,12.0,6867.0,6779.0,0.0,0.226586,0.189542,0.188009
1302202,13381,34,26,0,49,3,2015,14.0,431.0,818.0,19.0,0.207101,0.071451,0.148434
583504,5907,19,28,0,20,5,2015,0.0,1459.0,7130.0,131.0,0.071023,0.246038,0.711720
2004411,20604,26,25,0,72,2,2015,11.0,1339.0,1875.0,9.0,0.181009,0.204092,0.142271


In [41]:
#did not use train/validation to save time since overfitting was much less of a concern

from sklearn.linear_model import ElasticNet, Lasso, Ridge, LinearRegression,BayesianRidge, HuberRegressor

features = [
    'item_cnt_block_avg', 
    'shop_cnt_block_avg',
    'category_cnt_block_avg',
    'shop_category_cnt_block_avg'
]

lr_model =  LinearRegression()
x_train = consolidated[['date_block_num']+features]
y_train = consolidated['shop_item_cnt_block']
lr_model.fit(x_train, y_train)

from sklearn.metrics import mean_squared_error
from math import sqrt

train_preds = lr_model.predict(x_train)
train_preds.clip(0,20,out=train_preds)
rms = sqrt(mean_squared_error(y_train, train_preds))
print("rmse: ", rms)

rmse:  0.9658089636255409


In [42]:
lr_model.coef_

array([ 0.        , 26.9486764 ,  0.51294499, -0.17438215,  1.74442185])

In [48]:
test.head()

,ID,shop_id,item_id,item_cnt_block_avg,shop_cnt_block_avg,category_cnt_block_avg,shop_category_cnt_block_avg
0,0,5,5037,0.020176,0.161951,0.394336,0.060041
1,1,5,5320,0.000000,0.161951,0.696913,0.078711
2,2,5,5233,0.018588,0.161951,0.394336,0.060041
3,3,5,5232,0.006189,0.161951,0.336978,0.039470
4,4,5,5268,0.000000,0.161951,0.625360,0.052231


In [47]:
len(test)

214200

In [46]:
test = test.merge(consolidated[cols + features],on=cols,how='left')

In [49]:
test['date_block_num'] = 34

In [50]:
preds = lr_model.predict(test[['date_block_num']+features])
preds.clip(0,20,out=preds)

array([0.67712606, 0.11322301, 0.63434437, ..., 0.08568678, 0.01568121,
       0.08189779])

In [51]:

print(np.mean(preds))
print(np.max(preds))

0.255489309431921
20.0


In [52]:
submission = test.loc[:,['ID']]
submission['item_cnt_month'] = preds

submission.to_csv('submission.csv', index=False)